In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['OMP_NUM_THREADS'] = '4'
import gc

path = 'input_data/'
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }


In [3]:
print('load train....')
train_df = pd.read_csv(path+"train_sampling5.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])
len_train = len(train_df)

print('load test_sample5....')
test5_df = pd.read_csv(path+"test_sampling5.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])
len_test5 = len(test5_df)
train_df=train_df.append(test5_df)
len_test5 = len(train_df)
del test5_df; gc.collect()

print('load test....')
test_df = pd.read_csv(path+"kaggle_test.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])
train_df=train_df.append(test_df)
del test_df; gc.collect()

print(len_train, len_test5,len(train_df))
print("sample # of val:", len_test5-len_train)
print("sample # of test:",len(train_df)-len_test5)

load train....
load test_sample5....
load test....
(8620160, 27110549, 45901018)
('sample # of val:', 18490389)
('sample # of test:', 18790469)


In [4]:
print('hour, day, wday....')
train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')
train_df['wday']  = pd.to_datetime(train_df.click_time).dt.dayofweek.astype('uint8')
print('grouping by ip-day-hour combination....')
gp = train_df[['ip','day','hour','channel']].groupby(by=['ip','day','hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'qty'})
train_df = train_df.merge(gp, on=['ip','day','hour'], how='left')
del gp; gc.collect()
print('group by ip-app combination....')
gp = train_df[['ip','app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_count'})
train_df = train_df.merge(gp, on=['ip','app'], how='left')
del gp; gc.collect()
print('group by ip-app-os combination....')
gp = train_df[['ip','app', 'os', 'channel']].groupby(by=['ip', 'app', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_os_count'})
train_df = train_df.merge(gp, on=['ip','app', 'os'], how='left')
del gp; gc.collect()
print("vars and data type....")
train_df['qty'] = train_df['qty'].astype('uint16')
train_df['ip_app_count'] = train_df['ip_app_count'].astype('uint16')
train_df['ip_app_os_count'] = train_df['ip_app_os_count'].astype('uint16')
print("label encoding....")
from sklearn.preprocessing import LabelEncoder
train_df[['app','device','os', 'channel', 'hour', 'day', 'wday']].apply(LabelEncoder().fit_transform)
print ('final part of preparation....')

test_df = train_df[len_test5:]
val_df = train_df[len_train:len_test5]
train_df = train_df[:len_train]
y_train = train_df['is_attributed'].values
y_val = val_df['is_attributed'].values
train_df.drop(['click_id', 'click_time','ip','is_attributed'],1,inplace=True)
val_df.drop(['click_id', 'click_time','ip','is_attributed'],1,inplace=True)


print("check-out:", len(y_train),len(y_val))
print("PRINT DATA")
print(train_df.head())

hour, day, wday....
grouping by ip-day-hour combination....
group by ip-app combination....
group by ip-app-os combination....
vars and data type....
label encoding....
final part of preparation....
('check-out:', 8620160, 18490389)
PRINT DATA
   app  channel  device  os  hour  day  wday   qty  ip_app_count  \
0   12      245       1  27     0    9     3    13           124   
1    2      477       1  15    12    7     1    14            76   
2    2      477       2   4    16    7     1    99          1007   
3   26      121       1   1     6    7     1     9             6   
4    3      153       1  11    16    8     2  2664         37493   

   ip_app_os_count  
0               22  
1                6  
2                9  
3                1  
4              285  


In [5]:
print ('neural network....')
from keras.layers import Input, Embedding, Dense, Flatten, Dropout, concatenate
from keras.layers import BatchNormalization, SpatialDropout1D, Conv1D
from keras.callbacks import Callback
from keras.models import Model
from keras.optimizers import Adam


neural network....


Using TensorFlow backend.


In [6]:
max_app = np.max([train_df['app'].max(),val_df['app'].max(), test_df['app'].max()])+1
max_ch = np.max([train_df['channel'].max(),val_df['channel'].max(), test_df['channel'].max()])+1
max_dev = np.max([train_df['device'].max(), val_df['device'].max(), test_df['device'].max()])+1
max_os = np.max([train_df['os'].max(), val_df['os'].max(), test_df['os'].max()])+1
max_h = np.max([train_df['hour'].max(), val_df['hour'].max(), test_df['hour'].max()])+1
max_d = np.max([train_df['day'].max(),val_df['day'].max(), test_df['day'].max()])+1
max_wd = np.max([train_df['wday'].max(),val_df['wday'].max(), test_df['wday'].max()])+1
max_qty = np.max([train_df['qty'].max(),val_df['qty'].max(), test_df['qty'].max()])+1
max_c1 = np.max([train_df['ip_app_count'].max(),val_df['ip_app_count'].max(), test_df['ip_app_count'].max()])+1
max_c2 = np.max([train_df['ip_app_os_count'].max(),val_df['ip_app_os_count'].max(), test_df['ip_app_os_count'].max()])+1

In [7]:
def get_keras_data(dataset):
    X = {
        'app': np.array(dataset.app),
        'ch': np.array(dataset.channel),
        'dev': np.array(dataset.device),
        'os': np.array(dataset.os),
        'h': np.array(dataset.hour),
        'd': np.array(dataset.day),
        'wd': np.array(dataset.wday),
        'qty': np.array(dataset.qty),
        'c1': np.array(dataset.ip_app_count),
        'c2': np.array(dataset.ip_app_os_count)
    }
    return X
train_df = get_keras_data(train_df)
val_df = get_keras_data(val_df)

In [8]:
emb_n = 50
dense_n = 1000
in_app = Input(shape=[1], name = 'app')
emb_app = Embedding(max_app, emb_n)(in_app)
in_ch = Input(shape=[1], name = 'ch')
emb_ch = Embedding(max_ch, emb_n)(in_ch)
in_dev = Input(shape=[1], name = 'dev')
emb_dev = Embedding(max_dev, emb_n)(in_dev)
in_os = Input(shape=[1], name = 'os')
emb_os = Embedding(max_os, emb_n)(in_os)
in_h = Input(shape=[1], name = 'h')
emb_h = Embedding(max_h, emb_n)(in_h) 
in_d = Input(shape=[1], name = 'd')
emb_d = Embedding(max_d, emb_n)(in_d) 
in_wd = Input(shape=[1], name = 'wd')
emb_wd = Embedding(max_wd, emb_n)(in_wd) 
in_qty = Input(shape=[1], name = 'qty')
emb_qty = Embedding(max_qty, emb_n)(in_qty) 
in_c1 = Input(shape=[1], name = 'c1')
emb_c1 = Embedding(max_c1, emb_n)(in_c1) 
in_c2 = Input(shape=[1], name = 'c2')
emb_c2 = Embedding(max_c2, emb_n)(in_c2) 
fe = concatenate([(emb_app), (emb_ch), (emb_dev), (emb_os), (emb_h), 
                 (emb_d), (emb_wd), (emb_qty), (emb_c1), (emb_c2)])
s_dout = SpatialDropout1D(0.2)(fe)
fl1 = Flatten()(s_dout)
conv = Conv1D(100, kernel_size=4, strides=1, padding='same')(s_dout)
fl2 = Flatten()(conv)
concat = concatenate([(fl1), (fl2)])
x = Dropout(0.2)(Dense(dense_n,activation='relu')(concat))
x = Dropout(0.2)(Dense(dense_n,activation='relu')(x))
outp = Dense(1,activation='sigmoid')(x)
model = Model(inputs=[in_app,in_ch,in_dev,in_os,in_h,in_d,in_wd,in_qty,in_c1,in_c2], outputs=outp)


In [11]:
batch_size = 50000 #10240
epochs = 2
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(list(train_df)[0]) / batch_size) * epochs
lr_init, lr_fin = 0.002, 0.0002
lr_decay = exp_decay(lr_init, lr_fin, steps)
optimizer_adam = Adam(lr=0.002, decay=lr_decay)
model.compile(loss='binary_crossentropy',optimizer=optimizer_adam,metrics=['accuracy'])

model.summary()

class_weight = {0:.5,1:.5} # magic
model.fit(train_df, y_train, batch_size=batch_size, validation_data=(val_df,y_val),epochs=2, class_weight=class_weight, shuffle=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
app (InputLayer)                (None, 1)            0                                            
__________________________________________________________________________________________________
ch (InputLayer)                 (None, 1)            0                                            
__________________________________________________________________________________________________
dev (InputLayer)                (None, 1)            0                                            
__________________________________________________________________________________________________
os (InputLayer)                 (None, 1)            0                                            
__________________________________________________________________________________________________
h (InputLa

In [12]:
print("Saved model to disk")
json_string = model.to_json()
with open("saved_models/model5.json", "w") as json_file:
    json_file.write(json_string)
model.save_weights('saved_models/weights5.h5')

Saved model to disk


In [ ]:
sub = pd.DataFrame()
sub['click_id'] = test_df['click_id'].astype('int')
test_df.drop(['click_id', 'click_time','ip','is_attributed'],1,inplace=True)
test_df = get_keras_data(test_df)

print("predicting....")
sub['is_attributed'] = model.predict(test_df, batch_size=batch_size)

del test_df; gc.collect()


predicting....


In [ ]:
import matplotlib.pyplot as plt

plt.hist(sub['is_attributed'])

In [ ]:
print("writing....")
sub.to_csv('sub_result/result_code5_sample5_emoji.csv',index=False)


In [ ]:
plt.hist(sub['is_attributed'])

In [ ]:

from sklearn import metrics
pred = model.predict(val_df, batch_size=batch_size)
fpr, tpr, thresholds = metrics.roc_curve(y_val, pred, pos_label=2)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.001, 1])
plt.ylim([0, 1.001])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show();